In [11]:
import pandas as pd

df = pd.read_csv("./Goodreads-Books.csv")

In [13]:
df.head()

,url,id,name,author,star_rating,num_ratings,num_reviews,summary,genres,first_published,about_author,community_reviews,kindle_price
0,https://www.goodreads.com/book/show/1047836.Ho...,1047836.Horror_Film_Directors_1931_1990,"Horror Film Directors, 1931-1990","[""Dennis Fischer""]",4.29,7.0,NaN,An exhaustive study of the major directors of ...,NaN,11/1/1991,"{""name"":""Dennis Fischer"",""num_books"":14}","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN
1,https://www.goodreads.com/book/show/4089333-au...,4089333-australian-urban-planning,"Australian Urban Planning: New Challenges, New...","[""Brendan Gleeson""]",3.00,1.0,NaN,"Designed for use by academics, students, plann...",NaN,2/1/2000,"{""name"":""Brendan Gleeson"",""num_books"":28}","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN
2,https://www.goodreads.com/book/show/26764878-m...,26764878-morgen-ohne-gestern,Morgen ohne gestern: Roman,"[""Regina Nössler""]",3.00,2.0,NaN,Christine Hoffmann wacht eines Morgens im Kran...,NaN,9/18/2015,"{""name"":""Regina Nössler"",""num_books"":28,""num_f...","{""1_stars"":{""reviews_num"":1,""reviews_percentag...","""$10.89"""
3,https://www.goodreads.com/book/show/23656944-z...,23656944-zen-and-the-art-of-recording,Zen and the Art of Recording,"[""Mixerman""]",4.19,88.0,4.0,"In this book, the third in the Zen and the Art...","[""Music"",""Nonfiction""]",10/1/2014,"{""name"":""Mixerman"",""num_books"":15,""num_followe...","{""1_stars"":{""reviews_num"":0,""reviews_percentag...","""$24.09"""
4,https://www.goodreads.com/book/show/26761586-t...,26761586-the-big-book-of-codewords,The Big Book Of Codewords,"[""Parragon Books""]",4.50,6.0,NaN,500 codeword puzzles in one great book!,NaN,9/5/2015,"{""name"":""Parragon Books"",""num_books"":6068,""num...","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN


In [15]:
df.sample(20)

,url,id,name,author,star_rating,num_ratings,num_reviews,summary,genres,first_published,about_author,community_reviews,kindle_price
4689167,https://www.goodreads.com/book/show/7020462-pl...,7020462-pleasure-of-the-heart-and-other-stories,Pleasure of the Heart and Other Stories,"[""K.G. McAbee""]",4.50,4.0,NaN,Thirteen award-winning short romance stories i...,"[""Anthologies""]",3/1/2002,"{""name"":""K.G. McAbee"",""num_books"":142,""num_fol...","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN
3428623,https://www.goodreads.com/book/show/10279010-o...,10279010-oodles-of-zoo-balloons,Oodles of Zoo Balloons,"[""Joelle Murphy""]",3.89,9.0,NaN,Ooh! Patrick has too many balloons. He has ood...,"[""Childrens""]",9/1/2009,"{""name"":""Joelle Murphy"",""num_books"":19,""num_fo...","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN
6267935,https://www.goodreads.com/book/show/18597901-t...,18597901-the-secret-of-zoe,The Secret of Zoe,"[""C.J. Renaud""]",3.42,12.0,NaN,"Zoe wants to be like other girls, ordinary gir...",NaN,9/7/2013,"{""name"":""C.J. Renaud"",""num_books"":1,""num_follo...","{""1_stars"":{""reviews_num"":2,""reviews_percentag...","""$2.99"""
1028130,https://www.goodreads.com/book/show/10338292-k...,10338292-kiss-of-the-wolf,Kiss of the Wolf,"[""Robyn Wren""]",4.33,15.0,NaN,\nKara moved through the darkness of the fores...,NaN,7/20/2007,"{""name"":""Robyn Wren"",""num_books"":6}","{""1_stars"":{""reviews_num"":0,""reviews_percentag...","""$2.99"""
6281467,https://www.goodreads.com/book/show/9636075-an...,9636075-an-analytical-approach-the-once-and-fu...,An Analytical Approach-The Once and Future King,"[""Raja Sharma""]",2.00,1.0,NaN,An Analytical Approach-The Once and Future Kin...,NaN,10/21/2010,"{""name"":""Raja Sharma"",""num_books"":387,""num_fol...","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN
4625426,https://www.goodreads.com/book/show/22669564-s...,22669564-spiritual-democracy,Spiritual Democracy: The Wisdom of Early Ameri...,"[""Steven B. Herrmann""]",4.80,5.0,1.0,"Exploring what the author calls the ""shaman-po...",NaN,10/14/2014,"{""name"":""Steven B. Herrmann"",""num_books"":4,""nu...","{""1_stars"":{""reviews_num"":0,""reviews_percentag...","""$10.99"""
3042427,https://www.goodreads.com/book/show/10119489-i...,10119489-images-of-christ,Images of Christ,"[""Anton de Gruchy""]",4.00,2.0,NaN,This is a 28 days study through the Gospel of ...,NaN,12/14/2010,"{""name"":""Anton de Gruchy"",""num_books"":2}","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN
2059151,https://www.goodreads.com/book/show/28573726-s...,28573726-schulen-und-studium-im-sozialen-wande...,Schulen und Studium im sozialen Wandel des hoh...,"[""Professor of Medieval History Johannes Fried""]",0.00,0.0,NaN,Papers presented at the meetings held in 1981 ...,NaN,12/31/1986,"{""name"":""Professor of Medieval History Johanne...","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN
4205845,https://www.goodreads.com/book/show/34438084-h...,34438084-how-to-develop-unshakeable-self-estee...,How to Develop Unshakeable Self-Esteem and Inc...,"[""Victor Cheng""]",4.43,7.0,1.0,NaN,NaN,NaN,"{""name"":""Victor Cheng"",""num_books"":13,""num_fol...","{""1_stars"":{""reviews_num"":0,""reviews_percentag...",NaN
5696851,https://www.goodreads.com/book/show/18588422-p...,18588422-psykhiatria-ve-mythos,Psykhiatria Ve Mythos,"[""Kriton Dinçmen""]",3.43,49.0,8.0,Psikiyatri Tıp'ın en baştan çıkarıcı alanı… Mi...,NaN,9/1/1997,"{""name"":""Kriton Dinçmen"",""num_books"":23,""num_f...","{""1_stars"":{""reviews_num"":3,""reviews_percentag...",NaN


In [19]:
import ast # Convert JSON-like strings in 'community_reviews' to dictionary format
df['community_reviews'] = df['community_reviews'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract 1-star reviews count and percentage
df['1_star_reviews_num'] = df['community_reviews'].apply(lambda x: x['1_stars']['reviews_num'] if isinstance(x, dict) else 0)
df['1_star_reviews_percentage'] = df['community_reviews'].apply(lambda x: x['1_stars']['reviews_percentage'] if isinstance(x, dict) else 0)

# Convert genres from string to list
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Expand genres column for analysis (explode so each genre is a separate row)
df_exploded = df.explode('genres')

In [25]:
df_exploded.head(30)

,url,id,name,author,star_rating,num_ratings,num_reviews,summary,genres,first_published,about_author,community_reviews,kindle_price,1_star_reviews_num,1_star_reviews_percentage
0,https://www.goodreads.com/book/show/1047836.Ho...,1047836.Horror_Film_Directors_1931_1990,"Horror Film Directors, 1931-1990","[""Dennis Fischer""]",4.29,7.0,NaN,An exhaustive study of the major directors of ...,NaN,11/1/1991,"{""name"":""Dennis Fischer"",""num_books"":14}","{'1_stars': {'reviews_num': 0, 'reviews_percen...",NaN,0,0
1,https://www.goodreads.com/book/show/4089333-au...,4089333-australian-urban-planning,"Australian Urban Planning: New Challenges, New...","[""Brendan Gleeson""]",3.00,1.0,NaN,"Designed for use by academics, students, plann...",NaN,2/1/2000,"{""name"":""Brendan Gleeson"",""num_books"":28}","{'1_stars': {'reviews_num': 0, 'reviews_percen...",NaN,0,0
2,https://www.goodreads.com/book/show/26764878-m...,26764878-morgen-ohne-gestern,Morgen ohne gestern: Roman,"[""Regina Nössler""]",3.00,2.0,NaN,Christine Hoffmann wacht eines Morgens im Kran...,NaN,9/18/2015,"{""name"":""Regina Nössler"",""num_books"":28,""num_f...","{'1_stars': {'reviews_num': 1, 'reviews_percen...","""$10.89""",1,50
3,https://www.goodreads.com/book/show/23656944-z...,23656944-zen-and-the-art-of-recording,Zen and the Art of Recording,"[""Mixerman""]",4.19,88.0,4.0,"In this book, the third in the Zen and the Art...",Music,10/1/2014,"{""name"":""Mixerman"",""num_books"":15,""num_followe...","{'1_stars': {'reviews_num': 0, 'reviews_percen...","""$24.09""",0,0
3,https://www.goodreads.com/book/show/23656944-z...,23656944-zen-and-the-art-of-recording,Zen and the Art of Recording,"[""Mixerman""]",4.19,88.0,4.0,"In this book, the third in the Zen and the Art...",Nonfiction,10/1/2014,"{""name"":""Mixerman"",""num_books"":15,""num_followe...","{'1_stars': {'reviews_num': 0, 'reviews_percen...","""$24.09""",0,0
4,https://www.goodreads.com/book/show/26761586-t...,26761586-the-big-book-of-codewords,The Big Book Of Codewords,"[""Parragon Books""]",4.50,6.0,NaN,500 codeword puzzles in one great book!,NaN,9/5/2015,"{""name"":""Parragon Books"",""num_books"":6068,""num...","{'1_stars': {'reviews_num': 0, 'reviews_percen...",NaN,0,0
5,https://www.goodreads.com/book/show/32471492-a...,32471492-antique-apples-from-kilcherman-s-chri...,Antique Apples from Kilcherman's Christmas Cov...,"[""Phyllis Bye Kilcherman""]",4.00,1.0,NaN,Antique Apples from Kilcherman’s Christmas Cov...,NaN,9/1/2016,"{""name"":""Phyllis Bye Kilcherman"",""num_books"":1}","{'1_stars': {'reviews_num': 0, 'reviews_percen...",NaN,0,0
6,https://www.goodreads.com/book/show/32476267-i...,32476267-it-s-a-wonderful-death,It's A Wonderful Death: A Fortannis Novel,"[""Derek Beebe""]",4.38,8.0,4.0,A brave knight breaks her vows to fight agains...,NaN,9/26/2016,"{""name"":""Derek Beebe"",""num_books"":7,""num_follo...","{'1_stars': {'reviews_num': 0, 'reviews_percen...",NaN,0,0
7,https://www.goodreads.com/book/show/4201989-an...,4201989-an-introduction-to-clay-colloid-chemistry,An Introduction to Clay Colloid Chemistry,"[""H. Van Olphen""]",4.50,2.0,NaN,NaN,NaN,5/12/1977,"{""name"":""H. Van Olphen"",""num_books"":10}","{'1_stars': {'reviews_num': 0, 'reviews_percen...",NaN,0,0
8,https://www.goodreads.com/book/show/4103828-om...,4103828-oman-under-qaboos,Oman Under Qaboos,"[""Calvin H. Allen""]",3.56,9.0,NaN,"This is an examination of the political, econo...",History,3/30/2000,"{""name"":""Calvin H. Allen"",""num_books"":4}","{'1_stars': {'reviews_num': 0, 'reviews_percen...","""$78.84""",0,0


In [27]:
#extract 5 star reviews
df_exploded['5_star_reviews_num'] = df['community_reviews'].apply(
    lambda x: x['5_stars']['reviews_num'] if isinstance(x, dict) else 0)
df_exploded['5_star_reviews_percentage'] = df['community_reviews'].apply(
    lambda x: x['5_stars']['reviews_percentage'] if isinstance(x, dict) else 0)

In [41]:
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10719469 entries, 0 to 6389858
Data columns (total 18 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   url                        object        
 1   id                         object        
 2   name                       object        
 3   author                     object        
 4   star_rating                float64       
 5   num_ratings                float64       
 6   num_reviews                float64       
 7   summary                    object        
 8   genres                     object        
 9   first_published            datetime64[ns]
 10  about_author               object        
 11  community_reviews          object        
 12  kindle_price               object        
 13  1_star_reviews_num         int64         
 14  1_star_reviews_percentage  int64         
 15  5_star_reviews_num         int64         
 16  5_star_reviews_percentage  int64        

In [31]:
import numpy as np

In [37]:
df_exploded['first_published'] = pd.to_datetime(df_exploded['first_published'], errors='coerce')
df_exploded['year_published'] = df_exploded['first_published'].dt.year  # Extract year only

In [39]:
df_exploded.sample(30)

,url,id,name,author,star_rating,num_ratings,num_reviews,summary,genres,first_published,about_author,community_reviews,kindle_price,1_star_reviews_num,1_star_reviews_percentage,5_star_reviews_num,5_star_reviews_percentage,year_published
2914316,https://www.goodreads.com/book/show/940412.The...,940412.The_Americanization_of_Edward_Bok,The Americanization of Edward Bok,"[""Edward William Bok""]",3.98,166.0,37.0,This Pulitzer Prize-winning autobiography char...,History,1920-01-01,"{""name"":""Edward William Bok"",""num_books"":54,""n...","{'1_stars': {'reviews_num': 6, 'reviews_percen...","""$4.95""",6,3,61,36,1920.0
4011113,https://www.goodreads.com/book/show/316552.The...,316552.The_Swedish_Table,The Swedish Table,"[""Helene Henderson""]",3.87,38.0,12.0,"""Although I never physically fit in . . . bein...",Scandinavian Literature,2005-01-01,"{""name"":""Helene Henderson"",""num_books"":21,""num...","{'1_stars': {'reviews_num': 0, 'reviews_percen...","""$9.99""",0,0,7,18,2005.0
1634404,https://www.goodreads.com/book/show/14379796-h...,14379796-how-to-write-music---musical-orthography,How to Write Music - Musical Orthography,"[""Clement A. Harris""]",3.50,2.0,NaN,"Many of the earliest books, particularly those...",NaN,2008-11-01,"{""name"":""Clement A. Harris"",""num_books"":4,""num...","{'1_stars': {'reviews_num': 0, 'reviews_percen...","""$0.00""",0,0,1,50,2008.0
5962961,https://www.goodreads.com/book/show/25318899-c...,25318899-convergence,Convergence: Superman (2015) #1,"[""Dan Jurgens""]",3.98,103.0,14.0,STARRING HEROES FROM THE PRE-FLASHPOINT DCU! A...,Comics,2015-04-08,"{""name"":""Dan Jurgens"",""num_books"":2312,""num_fo...","{'1_stars': {'reviews_num': 1, 'reviews_percen...","""$2.99""",1,0,30,29,2015.0
745348,https://www.goodreads.com/book/show/33790657-t...,33790657-the-ancient-secrets-of-a-royal-triad-...,The Ancient Secrets Of A Royal Triad Decoded,"[""Phalika Ngin""]",4.50,2.0,NaN,A casual walk in the park led me to finding th...,NaN,2013-01-01,"{""name"":""Phalika Ngin"",""num_books"":1,""num_foll...","{'1_stars': {'reviews_num': 0, 'reviews_percen...",NaN,0,0,1,50,2013.0
2000488,https://www.goodreads.com/book/show/21114134-t...,21114134-the-audition,The Audition,"[""Robert Muchamore""]",3.87,267.0,16.0,"Two kids, one band, one crucial audition. Noah...",Young Adult,2014-01-01,"{""name"":""Robert Muchamore"",""num_books"":126,""nu...","{'1_stars': {'reviews_num': 5, 'reviews_percen...",NaN,5,1,97,36,2014.0
815700,https://www.goodreads.com/book/show/74695709-1...,74695709-10-amazing-bridges,10 Amazing Bridges,"[""Chris Peacock""]",3.67,3.0,1.0,This quick-read guide introduces the reader to...,NaN,2011-04-25,"{""name"":""Chris Peacock"",""num_books"":12}","{'1_stars': {'reviews_num': 0, 'reviews_percen...","""$2.99""",0,0,1,33,2011.0
1526660,https://www.goodreads.com/book/show/25261029-l...,25261029-la-danza-del-amor-de-las-hadas-no-fic...,La danza del amor de las hadas (NO FICCIÓN),"[""Rosetta Forner""]",3.50,4.0,NaN,Tener cualidades maravillosas debería ser sufi...,NaN,2013-06-26,"{""name"":""Rosetta Forner"",""num_books"":54,""num_f...","{'1_stars': {'reviews_num': 0, 'reviews_percen...","""$9.99""",0,0,1,25,2013.0
2345538,https://www.goodreads.com/book/show/66427.Arou...,66427.Around_the_World_in_80_Hands,Around the World in 80 Hands,"[""Zia Mahmood""]",3.83,6.0,2.0,Taking you from world championship tournaments...,NaN,1999-10-01,"{""name"":""Zia Mahmood"",""num_books"":19}","{'1_stars': {'reviews_num': 0, 'reviews_percen...",NaN,0,0,1,16,1999.0
2256943,https://www.goodreads.com/book/show/6419502-d-...,6419502-d-n-angel-vol-13,"D.N.Angel, Vol. 13","[""Yukiru Sugisaki""]",4.15,892.0,17.0,"On his fourteenth birthday, Daisuke Niwa comes...",Manga,2008-01-01,"{""name"":""Yukiru Sugisaki"",""num_books"":138,""num...","{'1_stars': {'reviews_num': 3, 'reviews_percen...","""$4.99""",3,0,401,44,2008.0


In [46]:
df_exploded['num_reviews'] = df_exploded['num_reviews'].fillna(0).astype(int)
df_exploded['num_ratings'] = df_exploded['num_ratings'].fillna(0).astype(int)

In [49]:
#exploding community review column
df_exploded['2_star_reviews_num'] = df_exploded['community_reviews'].apply(
    lambda x: x['2_stars']['reviews_num'] if isinstance(x, dict) and '2_stars' in x else 0)
df_exploded['2_star_reviews_percentage'] = df_exploded['community_reviews'].apply(
    lambda x: x['2_stars']['reviews_percentage'] if isinstance(x, dict) and '2_stars' in x else 0)

df_exploded['3_star_reviews_num'] = df_exploded['community_reviews'].apply(
    lambda x: x['3_stars']['reviews_num'] if isinstance(x, dict) and '3_stars' in x else 0)
df_exploded['3_star_reviews_percentage'] = df_exploded['community_reviews'].apply(
    lambda x: x['3_stars']['reviews_percentage'] if isinstance(x, dict) and '3_stars' in x else 0)

df_exploded['4_star_reviews_num'] = df_exploded['community_reviews'].apply(
    lambda x: x['4_stars']['reviews_num'] if isinstance(x, dict) and '4_stars' in x else 0)
df_exploded['4_star_reviews_percentage'] = df_exploded['community_reviews'].apply(
    lambda x: x['4_stars']['reviews_percentage'] if isinstance(x, dict) and '4_stars' in x else 0)

In [61]:
# Drop columns
df_exploded = df_exploded.drop(columns=['community_reviews'])

df_exploded = df_exploded.drop(columns=['url'])

df_exploded = df_exploded.drop(columns=['id'])

df_exploded = df_exploded.drop(columns=['first_published'])

In [71]:
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10719469 entries, 0 to 6389858
Data columns (total 21 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   name                       object        
 1   author                     object        
 2   star_rating                float64       
 3   num_ratings                int64         
 4   num_reviews                int64         
 5   summary                    object        
 6   genres                     object        
 7   first_published            datetime64[ns]
 8   about_author               object        
 9   kindle_price               object        
 10  1_star_reviews_num         int64         
 11  1_star_reviews_percentage  int64         
 12  5_star_reviews_num         int64         
 13  5_star_reviews_percentage  int64         
 14  year_published             float64       
 15  2_star_reviews_num         int64         
 16  2_star_reviews_percentage  int64        

In [83]:
df_exploded.sample(30)

,name,author,star_rating,num_ratings,num_reviews,summary,genres,about_author,kindle_price,1_star_reviews_num,1_star_reviews_percentage,5_star_reviews_num,5_star_reviews_percentage,year_published,2_star_reviews_num,2_star_reviews_percentage,3_star_reviews_num,3_star_reviews_percentage,4_star_reviews_num,4_star_reviews_percentage
5438358,A Thousand Beginnings and Endings,Ellen Oh,3.77,5910,1376,Sixteen extraordinary authors—including New Yo...,Science Fiction,"{""name"":""Ellen Oh"",""num_books"":20,""num_followe...","""$11.99""",56,0,1138,19,2018.0,289,4,1752,29,2675,45
5295584,Historia Social del Mundo Occidental - del Feu...,Susana Bianchi,3.83,6,0,nan,nan,"{""name"":""Susana Bianchi"",""num_books"":6}",NaN,0,0,1,16,2005.0,1,16,0,0,4,66
3530044,California Fire and Life,Don Winslow,4.06,5001,328,"An insurance claims investigator, Jack Wade sp...",Mystery,"{""name"":""Don Winslow"",""num_books"":83,""num_foll...","""$13.99""",42,0,1649,32,1999.0,135,2,971,19,2204,44
3493914,Gossip in the First Decade of Victoria's Reign,John Ashton,4.00,4,2,This book was converted from its physical edit...,nan,"{""name"":""John Ashton"",""num_books"":233,""num_fol...","""$0.00""",0,0,2,50,1903.0,1,25,0,0,1,25
3625288,Suspense Stories Box Set,Parker Freeman,1.00,1,0,Suspense Stories Box Set (5 in 1) Detective No...,nan,"{""name"":""Parker Freeman"",""num_books"":9}",NaN,1,100,0,0,2016.0,0,0,0,0,0,0
20019,"The Superior Foes of Spider-Man, Vol. 2: The C...",Nick Spencer,3.87,838,78,The Foes are on the brink as the sleeper hit o...,Graphic Novels Comics,"{""name"":""Nick Spencer"",""num_books"":1101,""num_f...","""$8.99""",5,0,210,25,2014.0,43,5,217,25,363,43
4844677,Snorra-Edda: Ásamt Skáldu Og Þarmeð Fylgjandi ...,Rasmus Christian Rask,4.00,5,0,This is a reproduction of a book published bef...,nan,"{""name"":""Rasmus Christian Rask"",""num_books"":36}",NaN,0,0,1,20,2010.0,0,0,1,20,3,60
1851063,The Tanner Lectures on Human Values 1997,Thomas E. Hill,4.50,2,0,"The Tanner Lectures on Human Values, founded J...",Philosophy,"{""name"":""Thomas E. Hill"",""num_books"":21,""num_f...",NaN,0,0,1,50,1997.0,0,0,0,0,1,50
5804666,The Civilian Conservation Corps In Arizona's R...,Robert J. Moore,0.00,0,0,Part of the massive relief effort of Roosevelt...,nan,"{""name"":""Robert J. Moore"",""num_books"":85,""num_...",NaN,0,0,0,0,2006.0,0,0,0,0,0,0
3641590,Power Play,Jayne Castle,3.76,307,15,Blackmail! Talia Haywood could hardly believe ...,Contemporary,"{""name"":""Jayne Castle"",""num_books"":50,""num_fol...",NaN,9,2,94,30,1982.0,27,8,87,28,90,29


In [63]:
df_exploded['author'] = df_exploded['author'].apply(lambda x: x.replace('[', '').replace(']', '').replace('"', '').replace("'", "").strip())

In [67]:
# Remove rows where star_rating, num_ratings, and num_reviews are 0 and genres is NaN
df_exploded = df_exploded[~((df_exploded['star_rating'] == 0) & 
          (df_exploded['num_ratings'] == 0) & 
          (df_exploded['num_reviews'] == 0) & 
          (df_exploded['genres'].isna()))]

In [79]:
df_exploded['genres'].nunique()

1246

In [91]:
mask = (
    (df_exploded['star_rating'].fillna(0) == 0) &
    (df_exploded['num_ratings'].fillna(0) == 0) &
    (df_exploded['num_reviews'].fillna(0) == 0) &
    (
        df_exploded['genres'].isna() | 
        (df_exploded['genres'].str.lower() == 'nan') | 
        (df_exploded['genres'].str.strip() == '') ))

In [95]:
df_masked = df_exploded[mask]
print(df_masked)

                                                      name           author  \
11                                             塔木德大全集(珍藏版)              柯友辉   
16       Ok Cupid Dating Success: Attract & Seduce Girl...    Seventy Seven   
21       Workforce Housing: Innovative Strategies and B...  Richard Haughey   
27                                              Cosmopolis      PAUL BORGET   
33       The Koran, in 3 Hours: An Abridged, Unbiased A...     James   Dean   
...                                                    ...              ...   
6389827            Conversação libertária com Paulo Freire   Edson Passetti   
6389828                              70 Great Android Tips     Gaurav Dhaka   
6389832               The Tales of Adarya: Out of the Past  Alaina Buchwald   
6389843  The American Dream Gone Astray: Critical Reali...      Jürg Keller   
6389844      Storytelling: Selected Annotated Bibliography           Greene   

         star_rating  num_ratings  num_reviews  \
1

In [97]:
# View unique values in specific columns
unique_genres = df_masked['genres'].unique()
unique_star_rating = df_masked['star_rating'].unique()
unique_num_ratings = df_masked['num_ratings'].unique()
unique_num_reviews = df_masked['num_reviews'].unique()

print("Unique values in genres:", unique_genres)
print("Unique values in star_rating:", unique_star_rating)
print("Unique values in num_ratings:", unique_num_ratings)
print("Unique values in num_reviews:", unique_num_reviews)

Unique values in genres: ['nan']
Unique values in star_rating: [ 0. nan]
Unique values in num_ratings: [0]
Unique values in num_reviews: [0]


In [99]:
df_exploded = df_exploded[~mask]

In [105]:
#before
print("before shape:", df.shape)

#after
print("after shape:", df_exploded.shape)

before shape: (6389859, 21)
after shape: (10069926, 20)
